In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Creating Features from stock_price, financials, stock_list, options, Trades

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Stock Price

In [ ]:
df_TrainStockPrices = pd.read_csv("/kaggle/input/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv")
df_SuppStockPrices  =  pd.read_csv("/kaggle/input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv")
df_Price  = pd.concat([df_TrainStockPrices, df_SuppStockPrices])

In [ ]:
df_Price.shape[1]

In [ ]:
df_Price["SecuritiesCode"].nunique()

## Financials

In [ ]:
df_TrainFin = pd.read_csv("/kaggle/input/jpx-tokyo-stock-exchange-prediction/train_files/financials.csv")
df_SuppFin  =  pd.read_csv("/kaggle/input/jpx-tokyo-stock-exchange-prediction/supplemental_files/financials.csv")
df_fin  = pd.concat([df_TrainFin, df_SuppFin])

In [ ]:
df_fin["SecuritiesCode"].nunique()

### Filter only 2000 SecuritiesCodes

In [ ]:
list_2000 = df_Price["SecuritiesCode"].unique()
df_fin = df_fin[df_fin['SecuritiesCode'].isin(list_2000)].sort_values(by="SecuritiesCode")
df_fin["SecuritiesCode"].nunique()

### Creating features 

In [ ]:
base_df = df_Price.copy()

# Combine with financials
_financials = df_fin.copy()
_financials.rename(columns={'Date': 'Date_x', 'SecuritiesCode': 'SecuritiesCode_x'}, inplace=True)
base_df = base_df.merge(_financials, left_on='RowId', right_on='DateCode', how="left")

In [ ]:
base_df.shape[1]

## Stock list

In [ ]:
df_Stocklist = pd.read_csv("/kaggle/input/jpx-tokyo-stock-exchange-prediction/stock_list.csv")

In [ ]:
df_Stocklist["SecuritiesCode"].nunique()

### Filter only 2000 SecuritiesCodes

In [ ]:
df_Stocklist = df_Stocklist[df_Stocklist["Universe0"]==True]
df_Stocklist["SecuritiesCode"].nunique()

### Creating Features

In [ ]:
_stock_list = df_Stocklist.copy()
_stock_list.rename(columns={'Close': 'Close_x'}, inplace=True)
base_df = base_df.merge(_stock_list, on='SecuritiesCode', how="left")

In [ ]:
base_df.shape[1]

## Options

In [ ]:
df_TrainOptions = pd.read_csv("/kaggle/input/jpx-tokyo-stock-exchange-prediction/train_files/options.csv")
df_SuppOptions  =  pd.read_csv("/kaggle/input/jpx-tokyo-stock-exchange-prediction/supplemental_files/options.csv")
df_opt  = pd.concat([df_TrainOptions, df_SuppOptions])

### Filter only 2000 SecuritiesCodes

In [ ]:
extr = df_opt['OptionsCode'].astype('str').str.extract(r'^(\d{4})', expand=False).astype('int')
df_opt['SecuritiesCode'] = pd.to_numeric(extr)
df_opt = df_opt[df_opt['SecuritiesCode'].isin(list_2000)].sort_values(by="SecuritiesCode")
df_opt["SecuritiesCode"].nunique()

### Creating Features

In [ ]:
base_df = base_df.merge(df_opt, on=['Date', 'SecuritiesCode'], how="left")

In [ ]:
base_df.shape[1]

## Trades

In [ ]:
df_TrainTrades = pd.read_csv("/kaggle/input/jpx-tokyo-stock-exchange-prediction/train_files/trades.csv")
df_SuppTrades  =  pd.read_csv("/kaggle/input/jpx-tokyo-stock-exchange-prediction/supplemental_files/trades.csv")
df_Trades  = pd.concat([df_TrainTrades, df_SuppTrades])

In [ ]:
df_Trades = df_Trades.dropna(axis=0)

In [ ]:
df_Trades["Date"].unique().size

In [ ]:
print(df_Trades["Section"].unique())
print(df_Stocklist["NewMarketSegment"].unique())

In [ ]:
print(df_Trades[df_Trades["Section"]==df_Trades["Section"].unique()[0]].shape[0])
print(df_Trades[df_Trades["Section"]==df_Trades["Section"].unique()[1]].shape[0])
print(df_Trades[df_Trades["Section"]==df_Trades["Section"].unique()[2]].shape[0])

In [ ]:
df_Stocklist["Section/Products"].value_counts()

In [ ]:
df_Stocklist["NewMarketSegment"].value_counts()

### Creating Features

In [ ]:
_trades = df_Trades.copy()
_trades['NewMarketSegment'] = _trades['Section'].str.split(' \(', expand=True)[0]
base_df = base_df.merge(_trades, on=['Date', 'NewMarketSegment'], how="left")

In [ ]:
base_df.shape[1]

**Select your best features :)**